In [32]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
import json
from config import api_key

In [43]:
from census import Census
from us import states

c = Census(api_key, year=2018)

* As only civilians are legel to vote, it would make sense to only pull the civilian employment data.
* The labor force is the sum of employed and unemployed persons
* Columns data to pull: 
* Income variables: 
    "income": B19013_001E, "income_per_capita": B19301_001E
* Employment variables:
    "employable civilians": B23025_003E, "employed civilians": B23025_004E, "unemployed civilians": B23025_005E
* Population variables: 
    "Total population": B01003_001E
* Poverty variables:
    "poverty people": "B17001_002E"
* Age variables:
    "median age": "B01002_001E" 

In [46]:
raw_data = c.acs5.get(('NAME', 'B19013_001E', 'B19301_001E', 'B23025_003E',
                       'B23025_004E', 'B23025_005E', 'B01003_001E', "B01002_001E"), {'for':'state:*'})
df = pd.DataFrame(raw_data)
df = df.rename(columns = {'state': 'State#',
                          'NAME': 'State', 
                          'B01003_001E':'Total_population',
                          'B23025_003E':'Employable_civilians',
                          'B23025_004E':'Employed_civilians', 
                          'B23025_005E':'Unemployed_civilians', 
                          'B19013_001E':'Income',
                          'B19301_001E':'Income_per_capita',
                          "B01002_001E":'median age'
                          
    
})
df = df.sort_values('State#').reset_index(drop=True)
df

In [48]:
df

,State,Income,Income_per_capita,Employable_civilians,Employed_civilians,Unemployed_civilians,Total_population,median age,State#
0,Alabama,48486.0,26846.0,2224606.0,2076708.0,147898.0,4864680.0,38.9,01
1,Alaska,76715.0,35874.0,379219.0,351152.0,28067.0,738516.0,34.0,02
2,Arizona,56213.0,29265.0,3256192.0,3045978.0,210214.0,6946685.0,37.4,04
3,Arkansas,45726.0,25635.0,1368037.0,1292263.0,75774.0,2990671.0,37.9,05
4,California,71228.0,35021.0,19630514.0,18309012.0,1321502.0,39148760.0,36.3,06
5,Colorado,68811.0,36415.0,2978700.0,2839374.0,139326.0,5531141.0,36.6,08
6,Connecticut,76106.0,43056.0,1936354.0,1810445.0,125909.0,3581504.0,40.8,09
7,Delaware,65627.0,33989.0,479447.0,450946.0,28501.0,949495.0,40.2,10
8,District of Columbia,82604.0,53321.0,398567.0,369194.0,29373.0,684498.0,33.9,11
9,Florida,53267.0,30197.0,9876910.0,9253932.0,622978.0,20598139.0,41.9,12
